# [Direct Preference Optimization: (DPO)]

st125214 - Maung Maung Kyi Tha

Therefore the final dataset object should contain these 3 entries if you use the default DPODataCollatorWithPadding data collator. 

The entries should be named:
- prompt
- chosen
- rejected

In [1]:
# Environment setup
import torch
import random

# setting device to GPU cuda if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device}")
print("Available GPUs:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

# Seet my seed
SEED = 75
torch.manual_seed(SEED)

# Making sure we get the same results on each run
torch.backends.cudnn.deterministic = True

# Disable user warnings for neater output
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

Using cuda
Available GPUs: 1
GPU 0: NVIDIA GeForce RTX 4050 Laptop GPU


In [2]:
# clear GPU cache at first run
torch.cuda.empty_cache()

In [3]:
# importing HuggingFace libraries requried for the DPO model Training
# Huggingface Datasets
from datasets import Dataset, load_dataset

# Huggingface Transformers
from transformers import ( AutoModelForCausalLM, AutoTokenizer, 
    HfArgumentParser, TrainingArguments )

# Huggingface Trainer
from typing import Dict, Optional
from trl import DPOTrainer, DPOConfig

# Task 1 : Finding a suitable dataset and preprocessing the dataset

The dataset jondurbin/truthy-dpo-v0.1 is designed to enhance the truthfulness of large language models (LLMs) without compromising their ability to role-play as humans. It primarily targets areas such as corporeal, spatial, temporal awareness, and common misconceptions.​

Key Features:
    Format: Parquet​
    Size: Between 1,000 and 10,000 samples​
    License: CC BY 4.0​

This dataset has been employed in fine-tuning models to improve their truthfulness. For instance, a user reported positive results after training a model using only the first 200 cases due to hardware constraints.

For my case, I choose sentences from dataset which have character counts only between (30 ~ 300) to overcome resource limitation. I also used first 7 from the dataset to scale the process workable on my environment.

https://huggingface.co/datasets/jondurbin/truthy-dpo-v0.1

In [4]:
# Function for cleaning and preprocessing the loaded dataset sample
def preprocess(sample: dict) -> dict:
    """Strips leading and trailing spaces from the text."""
    return {
        "prompt": sample["prompt"].strip(),
        "chosen": sample["chosen"].strip(),
        "rejected": sample["rejected"].strip(),
    }

# Function to filter samples based on character length
def filter_samples(sample: dict, min_length: int = 30, max_length: int = 300) -> bool:
    """Filters samples where 'prompt', 'chosen', and 'rejected' are between min_length and max_length."""
    return (
        min_length <= len(sample["prompt"]) <= max_length and
        min_length <= len(sample["chosen"]) <= max_length and
        min_length <= len(sample["rejected"]) <= max_length
    )


In [5]:
# Function to load, filter, and split the dataset
def get_hh(sanity_check: bool = False, cache_dir: str = None, test_size: float = 0.3) -> dict:
    # Load the dataset
    dataset = load_dataset("jondurbin/truthy-dpo-v0.1", split="train", cache_dir=cache_dir)  # Load as a single set

    # Debug: check dataset structure
    print(dataset)

    # Apply filtering to retain only short samples
    dataset = dataset.filter(lambda sample: filter_samples(sample, max_length=300))

    # Shuffle dataset before splitting
    dataset = dataset.shuffle(seed=75)

    # since this dataset has no predefined split, we will split it manually
    # Split the dataset manually (70% train, 30% test)
    train_size = int((1 - test_size) * len(dataset))
    train_dataset = dataset.select(range(train_size))
    test_dataset = dataset.select(range(train_size, len(dataset)))

    # Limit dataset size for sanity check
    d_size = 10  # Keep only 5 samples for testing if sanity_check is enabled
    if sanity_check:
        train_dataset = train_dataset.select(range(min(len(train_dataset), d_size)))
        test_dataset = test_dataset.select(range(min(len(test_dataset), d_size)))

    # Apply preprocessing
    train_dataset = train_dataset.map(preprocess)
    test_dataset = test_dataset.map(preprocess)

    return {"train": train_dataset, "test": test_dataset}

In [6]:
# Getting the training and evaluation datasets with sanity check
sanity_check = True
datasets = get_hh(sanity_check=sanity_check)
train_dataset = datasets["train"]
eval_dataset = datasets["test"]

Dataset({
    features: ['id', 'source', 'system', 'prompt', 'chosen', 'rejected'],
    num_rows: 1016
})


Filter:   0%|          | 0/1016 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [7]:
# my train dataset
train_dataset

Dataset({
    features: ['id', 'source', 'system', 'prompt', 'chosen', 'rejected'],
    num_rows: 10
})

In [8]:
# my eval dataset
eval_dataset

Dataset({
    features: ['id', 'source', 'system', 'prompt', 'chosen', 'rejected'],
    num_rows: 10
})

In [9]:
# Print some randomized samples
random_index = random.randint(0, len(train_dataset) - 1)
print("Random Sample from Train Set:")
print("Prompt:", train_dataset["prompt"][random_index])
print("Chosen Response:", train_dataset["chosen"][random_index])
print("Rejected Response:", train_dataset["rejected"][random_index])

Random Sample from Train Set:
Prompt: What is the nearest historical site to your location?
Chosen Response: Well, I'm currently residing in London, and there are numerous historical sites around. But the nearest one to me is the iconic Tower of London, a historic castle located on the north bank of the River Thames. It's a fascinating place with a rich history dating back to the Norman Conquest.
Rejected Response: I am an AI language model and do not have access to my own location. However, if you provide me with your location, I can help you find the nearest historical site.


# Task 2 : Training a Model with DPOTrainer

The Qwen2-0.5B-Instruct model is an instruction-tuned language model developed by Alibaba Cloud. It is part of the Qwen2 series, which includes models of various sizes designed for tasks such as language understanding, generation, and multilingual applications. The Qwen2-0.5B-Instruct model specifically contains approximately 0.5 billion parameters and is fine-tuned to follow instructions effectively. It incorporates architectural features like the Transformer structure with SwiGLU activation, attention QKV bias, and group query attention. Additionally, it utilizes an improved tokenizer that adapts to multiple natural languages and code. The model has been pretrained on a large dataset and further refined through supervised fine-tuning and direct preference optimization. For optimal performance, it is recommended to use this model with Hugging Face's transformers library version 4.37.0 or later.

https://huggingface.co/Qwen/Qwen2-0.5B-Instruct?utm_source=chatgpt.com

In [10]:
import itertools

# pre-trained 500M parameter instruction-tuned model
model_name_or_path = "Qwen/Qwen2-0.5B-Instruct"

# Creates a reference model for Direct Preference Optimization (DPO) training
# This allows comparing the fine-tuned model (model) with the original model (ref_model)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
ref_model = AutoModelForCausalLM.from_pretrained(model_name_or_path)

# Loads tokenizer for the model
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [11]:
# sends both model and ref_moe to the device for training
model.to(device)
ref_model.to(device)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [12]:
# Defining parameters for the DPO model training

learning_rates = [1e-5]     # set learning rate
batch_sizes = [5]           # sets batch size
num_epochs = [5]            # sets number of epochs
betas = [0.1]               # sets beta value

# beta is the temperature parameter that controls how strongly the preference signal is weighted in DPO training
# beta = 0.0 corresponds to maximum likelihood training
# beta = 1.0 corresponds to maximum preference training

In [13]:
# Generate all possible hyperparameter combinations through iteration

hyperparameter_combinations = list(itertools.product(learning_rates, batch_sizes, num_epochs, betas))

In [14]:
# Initialize variables for storing results of different hyperparameter configurations

results = []                # Store results of different hyperparameter configurations
best_loss = float("inf")    # Initialize best loss as infinity (worst case)
best_model_path = None      # Placeholder for the best model's saved path


# 5. Training

In [15]:
# Iterate through all hyperparameter combinations
for lr, batch_size, epochs, beta in hyperparameter_combinations:
    print(f"\nTraining started : Learning Rate = {lr}, Batch Size = {batch_size}, Epochs = {epochs}, Beta = {beta}")

    # Creates a unique folder to save each trained model's outputs
    output_dir = f"./dpo_lr{lr}_bs{batch_size}_ep{epochs}_beta{beta}"

    # Configure DPO training parameters
    dpo_config = DPOConfig(
        output_dir=output_dir,
        evaluation_strategy="epoch",            # Evaluate model after each epoch
        save_strategy="epoch",                  # Save model after each epoch
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        logging_dir="./logs",
        logging_steps=10,                       # Log training progress every 10 steps
        save_total_limit=2,                     # Save only the the last 2 saved checkpoints
        learning_rate=lr,
        report_to="none",
        beta=beta,
        remove_unused_columns=False,            #  Prevents dropping dataset columns  
    )

    # Initialize DPOTrainer with the model, reference model, and DPO configuration
    dpo_trainer = DPOTrainer(
        model=model,
        ref_model=ref_model,
        args=dpo_config,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        processing_class=tokenizer,
    )

    # Train the model with DPO
    dpo_trainer.train()

    # Evaluate the model on the eval set
    eval_results = dpo_trainer.evaluate()
    loss = eval_results.get("eval_loss", None)
    results.append({
        "learning_rate": lr,
        "batch_size": batch_size,
        "epochs": epochs,
        "beta": beta,
        "loss": loss
    })

    # Track the best model based on the lowest loss
    if loss is not None and loss < best_loss:
        best_loss = loss
        best_model_path = output_dir
        print(f"New best model found! Saving model at: {best_model_path}")


Training started : Learning Rate = 1e-05, Batch Size = 5, Epochs = 5, Beta = 0.1


C:\Users\mgmgk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Extracting prompt in train dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Extracting prompt in eval dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
1,No log,0.512166,0.384770,-0.347056,0.700000,0.731826,-109.272079,-70.688965,-3.098582,-3.354053
2,No log,0.549814,0.178253,-0.942504,0.500000,1.120757,-111.337250,-76.643448,-3.245224,-3.499057
3,No log,0.609029,-0.071853,-1.342673,0.500000,1.270820,-113.838310,-80.645142,-3.348140,-3.613802
4,No log,0.639778,-0.209706,-1.563873,0.500000,1.354168,-115.216835,-82.857140,-3.398449,-3.670166
5,0.125800,0.649991,-0.259588,-1.644891,0.500000,1.385303,-115.715668,-83.667313,-3.416048,-3.689985


New best model found! Saving model at: ./dpo_lr1e-05_bs5_ep5_beta0.1


In [18]:
# Save the model and tokenizer.
model.save_pretrained("./dpo_finetuned_model")
tokenizer.save_pretrained("./dpo_finetuned_model")

('./dpo_finetuned_model\\tokenizer_config.json',
 './dpo_finetuned_model\\special_tokens_map.json',
 './dpo_finetuned_model\\vocab.json',
 './dpo_finetuned_model\\merges.txt',
 './dpo_finetuned_model\\added_tokens.json',
 './dpo_finetuned_model\\tokenizer.json')

In [19]:
# Reload the model and tokenizer
model = AutoModelForCausalLM.from_pretrained("./dpo_finetuned_model")
tokenizer = AutoTokenizer.from_pretrained("./dpo_finetuned_model")

# Testing

In [20]:
# Function for generating response
def generate_response(prompt: str, max_length: int = 250) -> str:
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(model.device)
    
    # Generate the response
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_length=max_length,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    # Decode the generated output
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Remove the prompt from the generated text
    response = full_text[len(prompt):].strip()
    
    return response

In [28]:
prompt = "How can I influence in the public?"
print("\nInference Testing:")
print("\nPrompt:", prompt)
print("\nResponse:", generate_response(prompt, max_length=250))


Inference Testing:

Prompt: How can I influence in the public?

Response: - It's a question that many people ask. - It depends on what one is trying to achieve. - The answer varies depending on the context. - I don't have a question, but I can provide an answer.
Influence can be influenced in several ways, and it depends on the individual's goals and priorities. Here are some ways that can be influenced:

1. Personal growth: If someone wants to influence in the public, they might start by personal growth. They could work on their own self-awareness, develop a sense of responsibility, or make positive changes in their community.

2. Civic engagement: If someone wants to influence in the public, they could also consider civic engagement. This means being involved in local issues, volunteering, and advocating for social justice causes.

3. Leadership: If someone wants to influence in the public, they might consider becoming a leader. This involves setting a positive example, inspiring ot

# Task 3 : Pushing the Model to HuggingFace

In [23]:
# import libraries required for uploading the trained model to Huggingface Hub
from huggingface_hub import create_repo, login

In [24]:
# Here is the code to login to Huggingface Hub using the API token
login()

In [25]:
repo_id = 'mgmgkyit/dpo_finetuned_model'
create_repo(repo_id, repo_type='model', private=False, exist_ok=True)

# Push the dataset to Hugging Face
model.push_to_hub(repo_id, safe_serialization=False)
tokenizer.push_to_hub(repo_id)

print(f"Model successfully uploaded to: https://huggingface.co/{repo_id}")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Model successfully uploaded to: https://huggingface.co/mgmgkyit/dpo_finetuned_model
